In [ ]:
import random
from math import floor, exp, sqrt, pi, cos, sin

In [6]:
# Установка библиотеки sklearn
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 8.3/8.3 MB 695.8 kB/s eta 0:00:00
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
     -------------------------------------- 40.2/40.2 MB 554.5 kB/s eta 0:00:00


In [3]:
from sklearn.linear_model import LinearRegression

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def round_to_2(x):    
    return round(x, 2)

# Grid search

In [ ]:
def score_model(model, x_val, y_val):
   
    y_pred = model.predict(x_val)
    
    res = 0

    for i in range(len(y_val)):
        res += abs(y_pred[i] - y_val[i])
        
    return res / len(y_val)

In [ ]:
def make_featured_data(config,data):
    new_data = []
    for i in range(len(data)):
        row = []
        for feature in config:
            row.append(data[i][feature])
        new_data.append(row)
    return new_data

def grid_search_solution(
        model,
        model_configs,
        x_train, y_train,
        x_val, y_val):
    """
    Принимает набор конфигураций модели линейной регрессии и находит лучшую из них
    с помощью алгоритма grid search.
    Для оценки точности модели на валидационной выборке используется функция score_model.

    Аргументы:
        model: Модель.
        model_configs: Список конфигураций модели линейной регрессии.
                       Каждая конфигурация — список номеров факторов,
                       которые используются для обучения модели.
        x_train: Список объектов обучающей выборки.
        y_train: Список значений предсказываемой характеристики для объектов из обучающей выборки.
                 Значение на $i$-ой позиции в списке соответствует $i$-ому объекту обучающей выборки.
        x_val: Список объектов валидационной выборки.
        y_val: Список значений предсказываемой характеристики для объектов из валидационной выборки.
               Значение на $i$-ой позиции в списке соответствует $i$-ому объекту валидационной выборки.

    Возвращаемое значение:
        Возвращает пару значений: лучшая конфигурация, точность модели с данной конфигурацией
        на валидационной выборке.
    """
    max_cfg = ([-1], 9999)
    for config in model_configs:
        # model.set_params(model_configs[0][0])
        x_featured = make_featured_data(config,x_train)
        model.fit(x_featured, y_train)
        x_val_featured = make_featured_data(config,x_val)
        if (score := score_model(model,x_val_featured , y_val)) < max_cfg[1]:
            max_cfg = (config, score)
    max_cfg = (max_cfg[0], round_to_2(max_cfg[1]))
    return max_cfg

In [ ]:
def process_data(file_path, frac=0.9):
    data_x_y = []
    
    with open(file_path) as f:
        cnt = 0
        
        for line in f:
            if cnt == 0:
                cnt += 1
                continue
                
            l = line.strip().split(',')
            data_x_y.append(([float(x) for x in l[:-1]], float(l[-1])))  
            
            cnt += 1
            
    cut_n = floor(frac * len(data_x_y))
            
    train_x_y = data_x_y[:cut_n]
    val_x_y = data_x_y[cut_n:]
    
    train_x, train_y = [], []
    
    for x, y in train_x_y:
        train_x.append(x)
        train_y.append(y)
        
    val_x, val_y = [], []
    
    for x, y in val_x_y:
        val_x.append(x)
        val_y.append(y)
    
    return train_x, train_y, val_x, val_y

def grid_search_test():
    def split_df_to_xs_and_ys(df):
        xs = []
        ys = []
        
        for _, row in df.iterrows():
            xs.append([row['x1'], row['x2'], row['x3'], row['x4']])
            ys.append(row['y'])
            
        return xs, ys
        
    data_x_train_example_1 = [[1, -33], [2, -21], [3, -34234]]
    data_y_train_example_1 = [1, 2, 3]
    
    data_x_val_example_1 = [[4, -231], [5, -342341], [6, -23]]
    data_y_val_example_1 = [4, 5, 6]
    
    configs_example_1 = [[0], [1]]
    
    res_example_1 = ([0], 0)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_1,
                                data_x_train_example_1, data_y_train_example_1,
                                data_x_val_example_1, data_y_val_example_1) == res_example_1
    
    data_x_train_example_2, data_y_train_example_2, \
        data_x_val_example_2, data_y_val_example_2 = process_data('grid_search_test_data.csv', frac=0.9)
    
    configs_example_2 = [[0], [0, 1], [0, 1, 2], [2, 3], [1]]
    
    res_example_2 = ([0, 1, 2], 0.26)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_2,
                                data_x_train_example_2, data_y_train_example_2,
                                data_x_val_example_2, data_y_val_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [ ]:
grid_search_test()

# Метод дихотомии

In [ ]:
def dichotomous_search_solution(f, a, b, eps):
    """
    Производит поиск минимума заданной функции в заданном интервале с помощью метода дихотомии.
    
    Аргументы:
        f: Функция от одного аргумента, минимум которой необходимо найти.
        a: Левая граница интервала, в котором происходит поиск минимума.
        b: Правая граница интервала, в котором происходит поиск минимума.
        eps: Допустимая погрешность при поиске минимума.
        
    Возвращаемое значение:
        Возвращает координату точки, в которой достигается минимальное значение функции.
        Координата должна быть округлена до 2 знаков после запятой.
    """
    
    pass

In [ ]:
def dichotomous_search_test():
    f_example_1 = lambda x: x ** 2
    
    a_example_1 = -2
    b_example_1 = 5
    eps_example_1 = 0.01
    
    res_example_1 = 0
    
    assert dichotomous_search_solution(f_example_1, a_example_1, b_example_1, eps_example_1) == res_example_1
    
    f_example_2 = lambda x: (exp(2 * (x - 2)) + 1) * (x + 1) ** 2
    
    a_example_2 = -100
    b_example_2 = 200
    eps_example_2 = 0.01
    
    res_example_2 = -1
    
    assert dichotomous_search_solution(f_example_2, a_example_2, b_example_2, eps_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [ ]:
dichotomous_search_test()

# Градиентный спуск

In [ ]:
def gradient_descent_solution(grad_f, x_0, alpha, eps):
    """
    Производит поиск минимума заданной функции двух аргументов с помощью градиентного спуска.
    
    Аргументы:
        grad_f: Градиент функции двух аргументов, для которой необходимо найти минимум.
                Функция принимает на вход точку (список из двух значений) и возвращает
                вектор градиента в этой точке (тоже список из двух значений).
        x_0: Стартовая точка (список из двух значений), из которой запускается алгоритм градиентного спуска.
        alpha: Коэффициент скорости обучения.
        eps: Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Координаты точки (список из двух значений), в которой достигается минимальное значение функции.
        Каждая координата должна быть округлена до 2 знаков после запятой.
    """
    
    pass

In [ ]:
def gradient_descent_test():
    f_example_1 = lambda x: x[0] ** 4 + x[1] ** 4
    grad_f_example_1 = lambda x: [4 * x[0], 4 * x[1]]
    
    x_0_example_1 = [32, -4]
    
    alpha_example_1 = 0.01
    eps_example_1 = 0.001
    
    res_example_1 = [0.02, 0]
    
    assert gradient_descent_solution(grad_f_example_1, x_0_example_1, alpha_example_1, eps_example_1) == res_example_1
    
    f_example_2 = lambda x: (x[0] - 1) ** 4 + 2 * (x[1] + 2) ** 2 - 1
    grad_f_example_2 = lambda x: [4 * (x[0] - 1) ** 3, 4 * (x[1] + 2)]
    
    x_0_example_2 = [-2, 2]
    
    alpha_example_2 = 0.0001
    eps_example_2 = 0.0001
    
    res_example_2 = [0.58, -1.76]
    
    assert gradient_descent_solution(grad_f_example_2, x_0_example_2, alpha_example_2, eps_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [ ]:
gradient_descent_test()

# Метод имитации отжига

In [ ]:
def g(x):
    return 2 * x ** 2 + cos(pi * x) - 2.9 * sin(2 * pi * x) + cos(3 * pi * x) * sin(pi * x)

In [ ]:
def sub_with_p(random_gen, p, new_x, x):
    """
    С заданной вероятностью возвращает точку `new_x`, в остальных случаях
    возвращает точку x.
    
    Аргументы:
        random_gen: Генератор случайных чисел.
        p: Вероятность, с которой нужно вернуть первую точку.
        new_x: Точка, которая возвращается с вероятностью p.
        x: Точка, которая возвращает в остальных случаях.
        
    Возвращаемое значение:
        Точка, которая с вероятностью p будет равна new_x. В противном случае точка будет равна x.
    """
    
    val = random_gen.random()
    
    if val > p:
        return x
    
    return new_x

In [ ]:
def simulated_annealing_solution(f, x_0, t_0, lam, eps, random_gen):
    """
    Производит поиск минимума функции с помощью метода имитации отжига.
    
    Аргументы:
        f: Функция, минимум которой необходимо найти.
        x_0: Стартовая точка, из которой начинается процесс поиска минимума.
        t_0: Начальная температура системы.
        lam: Коэффициент охлаждения системы на каждом шаге.
             Охлаждение происходит по правилу T = lam * T.
        eps: Когда температура становится меньше eps, метод имитации отжига останавливает свою работу.
        random_gen: Генератор случайных чисел.
        
    Возвращаемое значение:
        Точка, которую метод считает минимумом функции.
    """
    
    pass

In [ ]:
def simulated_annealing_test():
    weierstrass_function = lambda x: 10 + x ** 2 - 10 * cos(2 * pi * x)

    f_example_1 = weierstrass_function
    
    x_0_example_1 = 20
    t_0_example_1 = 1000
    lam_example_1 = 0.99
    eps_example_1 = 0.001
    
    random_gen_example_1 = random.Random(0)
    
    res_example_1 = 0.0
    
    found_min_example_1 = \
        simulated_annealing_solution(weierstrass_function,
                                     x_0_example_1,
                                     t_0_example_1,
                                     lam_example_1,
                                     eps_example_1,
                                     random_gen_example_1)
    
    assert round_to_2(weierstrass_function(found_min_example_1)) == res_example_1
    
    print('Тест прошёл успешно!')

In [ ]:
simulated_annealing_test()